In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.autograd as autograd
from torch.autograd import Variable

from tqdm import tqdm
from time import time
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
####
import os
from os.path import dirname, realpath
import sys
sys.path.append(dirname(os.getcwd()))
import nltk as nk
from nltk.corpus import stopwords
import dill
import ConfigParser

In [15]:
import ConfigParser

config = ConfigParser.ConfigParser()
config.readfp(open(r'config.ini'))
ROOT_PATH = config.get('paths', 'root_path')
DATA_PATH = config.get('paths', 'data_path')
SAVE_PATH = config.get('paths', 'save_path')

MAX_TITLE_LEN = int(config.get('data_params', 'MAX_TITLE_LEN'))
MAX_BODY_LEN = int(config.get('data_params', 'MAX_BODY_LEN'))

print ROOT_PATH
print DATA_PATH
print SAVE_PATH
print MAX_TITLE_LEN
print MAX_BODY_LEN

/media/sf_vmsharefolder/courses/6.864_term_project/
data
docs
20
100


In [ ]:
np.random.seed(0)
#torch.manual_seed(0)

DATA_PATH = '../data/askubuntu/'

SAVE_PATH = './cnn_baseline.pt' 
EMBEDDINGS_FILE = DATA_PATH + '/vector/vectors_pruned.200.txt'
MAX_TITLE_LEN = 20
MAX_BODY_LEN = 100  #max number of words per sentence

tokenizer = RegexpTokenizer(r'\w+')

def get_embeddings():
    lines = []
    with open(EMBEDDINGS_FILE, 'r') as f:
        lines = f.readlines()
        f.close()
    
    embedding_tensor = []
    word_to_idx = {}
    
    for idx, l in enumerate(lines):
        word, emb = l.split()[0], l.split()[1:]
        vector = [float(x) for x in emb]
        if idx == 0: #reserved
            embedding_tensor.append(np.zeros(len(vector)))
        embedding_tensor.append(vector)
        word_to_idx[word] = idx+1
    #end for
    embedding_tensor = np.array(embedding_tensor, dtype=np.float32)    
    return embedding_tensor, word_to_idx
        
def get_tensor_idx(text, word_to_idx, max_len):
    null_idx = 0  #idx if word is not in the embeddings dictionary
    text_idx = [word_to_idx[x] if x in word_to_idx else null_idx for x in text][:max_len]
    if len(text_idx) < max_len:
        text_idx.extend([null_idx for _ in range(max_len - len(text_idx))])    
    x = torch.LongTensor(text_idx)  #64-bit integer
    return x
 

In [ ]:
#load data
print "loading data..."
print os.getcwd()
stop = stopwords.words('english')
tic = time()
train_text_file = DATA_PATH + 'texts_raw_fixed.txt'
train_text_df = pd.read_table(train_text_file, sep='\t', header=None)
train_text_df.columns = ['id', 'title', 'body']
train_text_df = train_text_df.dropna()
train_text_df['title_len'] = train_text_df['title'].apply(lambda words: len(tokenizer.tokenize(str(words))))
train_text_df['body_len'] = train_text_df['body'].apply(lambda words: len(tokenizer.tokenize(str(words))))
train_text_df['title'] = train_text_df['title'].apply(lambda words:' '.join(filter(lambda x: x not in stop,  words.split())))
train_text_df['body'] = train_text_df['body'].apply(lambda words: ' '.join(filter(lambda x: x not in stop,  words.split())))
train_text_df['title_no_stpwrds_len'] = train_text_df['title'].apply(lambda words: len(tokenizer.tokenize(str(words))))
train_text_df['body_no_stpwrds_len'] = train_text_df['body'].apply(lambda words: len(tokenizer.tokenize(str(words))))

train_idx_file = DATA_PATH + '/train_random.txt' 
train_idx_df = pd.read_table(train_idx_file, sep='\t', header=None)
train_idx_df.columns = ['query_id', 'similar_id', 'random_id']

dev_idx_file = DATA_PATH + '/dev.txt'
dev_idx_df = pd.read_table(dev_idx_file, sep='\t', header=None)
dev_idx_df.columns = ['query_id', 'similar_id', 'retrieved_id', 'bm25_score']
toc = time()
print "elapsed time: %.2f sec" %(toc - tic)

print "loading embeddings..."
tic = time()
embeddings, word_to_idx = get_embeddings()
print "vocab size (embeddings): ", len(word_to_idx)
toc = time()
print "elapsed time: %.2f sec" %(toc - tic)

In [ ]:
'''
stop = stopwords.words('english')
print stop
def gg(words, stop):
    return ' '.join(filter(lambda x: x not in stop,  words.split()))

ass = "how to get the your battery is broken message to go away ? everytime i turn on my computer , i see a message saying something like : your battery may be old or broken . etc ... i know my battery is not working . how can i make the message go away ?3	how can i set the software center to install software for non-root users ?	how can i set the software center to allow non-root users to install stuff from the ubuntu repos without having to type in their password ? i 'm fully aware of the security implications , and i am willing to take the risk . fedora 12 shipped with something like this . ( by modifying the policykit configuration , i believe )"
ass1 = "how to get the your battery is broken message to go away ? everytime i turn"
print gg(ass1,stop) 
'''

In [ ]:
all_words = []
all_words1 = []
count = 0
for sentence in train_text_df['title']:
    count += 1
    all_words += [word for word in tokenizer.tokenize(sentence) if word not in word_to_idx]

for sentence1 in train_text_df['body']:
    count += 1
    all_words1 += [word1 for word1 in tokenizer.tokenize(sentence) if word1 not in word_to_idx]
    

In [ ]:
print train_text_df.describe()

In [ ]:
#visualize data
f, ([ax1, ax2], [ax3, ax4]) = plt.subplots(2, 2)
sns.distplot(train_text_df['title_len'], hist=True, kde=True, color='b', label='title len', ax=ax1)
sns.distplot(train_text_df[train_text_df['body_len'] < 256]['body_len'], hist=True, kde=True, color='r', label='body len', ax=ax2)
sns.distplot(train_text_df['title_stopwords'], hist=True, kde=True, color='b', label='title len', ax=ax3)
sns.distplot(train_text_df['body_stopwords'], hist=True, kde=True, color='b', label='title len', ax=ax4)
ax1.axvline(x=MAX_TITLE_LEN, color='k', linestyle='--', label='max len')
ax2.axvline(x=MAX_BODY_LEN, color='k', linestyle='--', label='max len')
ax1.set_title('title length histogram'); ax1.legend(loc=1); 
ax2.set_title('body length histogram'); ax2.legend(loc=1);
plt.show()
#plt.savefig('question_len_hist.png')
print np.mean(train_text_df['body_len'])
print np.std(train_text_df['body_len'])
print np.mean(train_text_df['title_len'])
print np.std(train_text_df['title_len'])

print train_text_df.describe()

"""
print "fitting tf-idf vectorizer..."
tic = time()
tfidf = TfidfVectorizer(tokenizer=tokenizer.tokenize, analyzer='word', ngram_range=(1,1))
tfidf.fit(train_text_df['title'].tolist() + train_text_df['body'].tolist())
toc = time()
print "elapsed time: %.2f sec" %(toc - tic)

vocab = tfidf.vocabulary_
print "vocab size: ", len(vocab)
print "embeddings size: ", embeddings.shape
"""

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [20]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.autograd as autograd
from torch.autograd import Variable


In [16]:
idx = 0
query_id = train_idx_df.loc[idx, 'query_id']
similar_id_list = map(int, train_idx_df.loc[idx, 'similar_id'].split(' '))
random_id_list = map(int, train_idx_df.loc[idx, 'random_id'].split(' '))

print query_id
print similar_id_list
print random_id_list

query_title = train_text_df[train_text_df['id'] == query_id].title.tolist() 
query_body = train_text_df[train_text_df['id'] == query_id].body.tolist()
query_title_tokens = tokenizer.tokenize(query_title[0])[:MAX_TITLE_LEN]
query_body_tokens = tokenizer.tokenize(query_body[0])[:MAX_BODY_LEN]

print query_title_tokens
print query_body_tokens
'''

query_title_tensor_idx = get_tensor_idx(query_title_tokens, word_to_idx, MAX_TITLE_LEN) 
query_body_tensor_idx = get_tensor_idx(query_body_tokens, word_to_idx, MAX_BODY_LEN) 
'''

262144
[211039]
[227387, 413633, 113297, 356390, 256881, 145638, 296272, 318659, 86529, 48563, 53080, 65996, 334032, 517236, 470002, 177348, 502185, 248772, 457062, 339049, 265060, 264524, 15971, 40468, 422021, 177982, 513266, 437762, 318889, 163711, 491359, 72668, 523366, 513514, 119253, 167106, 395661, 411198, 284222, 75399, 352479, 250977, 476895, 384032, 379657, 47506, 214327, 295133, 165345, 278645, 40986, 147959, 498308, 328292, 49304, 518817, 418947, 155658, 522902, 316447, 487482, 100000, 436018, 235318, 314941, 187592, 439142, 26205, 180583, 418425, 41667, 188060, 147132, 68407, 191082, 455272, 486933, 229959, 153150, 101914, 254265, 314232, 384593, 366741, 227996, 384171, 302283, 335076, 452240, 144708, 395995, 221644, 392821, 197651, 404031, 319280, 257320, 38398, 287787, 83851]
['system', 'running', 'low', 'graphic', 'mode', 'ubuntu', 'without', 'monitor']
['ubuntu', 'desktop', '12', '10', 'want', 'desktop', 'server', 'ca', 'n', 't', 'start', 'without', 'monitor', 'tried', 

'\n\nquery_title_tensor_idx = get_tensor_idx(query_title_tokens, word_to_idx, MAX_TITLE_LEN) \nquery_body_tensor_idx = get_tensor_idx(query_body_tokens, word_to_idx, MAX_BODY_LEN) \n'

In [21]:
print "generating training, validation, test datasets..."
MAX_TITLE_LEN = int(MAX_TITLE_LEN)
MAX_BODY_LEN = int(MAX_BODY_LEN)
tic = time()
train_data = []
for idx in range(10):
#for idx in tqdm(range(train_idx_df.shape[0])):
    query_id = train_idx_df.loc[idx, 'query_id']
    similar_id_list = map(int, train_idx_df.loc[idx, 'similar_id'].split(' '))
    random_id_list = map(int, train_idx_df.loc[idx, 'random_id'].split(' '))
    
    #query title and body tensor ids
    query_title = train_text_df[train_text_df['id'] == query_id].title.tolist() 
    query_body = train_text_df[train_text_df['id'] == query_id].body.tolist()
    query_title_tokens = tokenizer.tokenize(query_title[0])[:MAX_TITLE_LEN]
    query_body_tokens = tokenizer.tokenize(query_body[0])[:MAX_BODY_LEN]
    query_title_tensor_idx = get_tensor_idx(query_title_tokens, word_to_idx, MAX_TITLE_LEN) 
    query_body_tensor_idx = get_tensor_idx(query_body_tokens, word_to_idx, MAX_BODY_LEN) 

    for similar_id in similar_id_list:
        sample = {}  #reset sample dictionary here
        sample['query_title'] = query_title_tensor_idx
        sample['query_body'] = query_body_tensor_idx

        similar_title = train_text_df[train_text_df['id'] == similar_id].title.tolist() 
        similar_body = train_text_df[train_text_df['id'] == similar_id].body.tolist()
        similar_title_tokens = tokenizer.tokenize(similar_title[0])[:MAX_TITLE_LEN]
        similar_body_tokens = tokenizer.tokenize(similar_body[0])[:MAX_BODY_LEN]
        similar_title_tensor_idx = get_tensor_idx(similar_title_tokens, word_to_idx, MAX_TITLE_LEN) 
        similar_body_tensor_idx = get_tensor_idx(similar_body_tokens, word_to_idx, MAX_BODY_LEN)
        sample['similar_title'] = similar_title_tensor_idx
        sample['similar_body'] = similar_body_tensor_idx

        for ridx, random_id in enumerate(random_id_list):
            random_title_name = 'random_title_' + str(ridx)
            random_body_name = 'random_body_' + str(ridx)
        
            random_title = train_text_df[train_text_df['id'] == random_id].title.tolist() 
            random_body = train_text_df[train_text_df['id'] == random_id].body.tolist()
            random_title_tokens = tokenizer.tokenize(random_title[0])[:MAX_TITLE_LEN]
            random_body_tokens = tokenizer.tokenize(random_body[0])[:MAX_BODY_LEN]
            random_title_tensor_idx = get_tensor_idx(random_title_tokens, word_to_idx, MAX_TITLE_LEN) 
            random_body_tensor_idx = get_tensor_idx(random_body_tokens, word_to_idx, MAX_BODY_LEN)
            sample[random_title_name] = random_title_tensor_idx
            sample[random_body_name] = random_body_tensor_idx
        #end for
        train_data.append(sample)
    #end for
#end for
toc = time()
print "elapsed time: %.2f sec" %(toc - tic)

generating training, validation, test datasets...
elapsed time: 2.66 sec


In [ ]:
#training parameters
num_epochs = 32 
batch_size = 8 

#model parameters
embed_dim = embeddings.shape[1] #200
hidden_dim = embed_dim / 2 
weight_decay = 1e-3 
learning_rate = 1e-3 

#RNN architecture
class RNN(nn.Module):
    
    def __init__(self, embed_dim, hidden_dim, vocab_size):
        super(RNN, self).__init__()
        self.hidden_dim = hidden_dim

        self.embedding_layer = nn.Embedding(vocab_size, embed_dim)
        self.embedding_layer.weight.data = torch.from_numpy(embeddings)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=32, batch_first=False)
        #TODO: check number of cells
    
    def init_hidden(self):
        #[num_layers, minibatch_size, hidden_dim]
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))

    def forward(self, x_idx):
        all_x = self.embedding_layer(x_idx)   
        lstm_out, self.hidden = self.lstm(all_x.view(len(x_idx),1,-1), self.hidden)
        return self.hidden 
        
use_gpu = torch.cuda.is_available()

model = RNN(embed_dim, hidden_dim, len(word_to_idx))
if use_gpu:
    print "found CUDA GPU..."
    model = model.cuda()
    
print model

#define loss and optimizer
criterion = nn.MarginRankingLoss(margin=1, size_average=True)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

training_loss = []
validation_loss = []

print "training..."
for epoch in range(num_epochs):
    
    running_train_loss = 0.0
    
    train_data_loader = torch.utils.data.DataLoader(
        train_data, 
        batch_size = batch_size,
        shuffle = True,
        num_workers = 4, 
        drop_last = True)
        
    model.train()
        
    for batch in tqdm(train_data_loader):
      
        #TODO: reshape to batches in the middle or use batch_first=True!
        query_title = Variable(batch['query_title'])
        query_body = Variable(batch['query_body'])
        similar_title = Variable(batch['similar_title'])
        similar_body = Variable(batch['similar_body'])
        #TODO: for loop over random_title and random_body

        if use_gpu:
            query_title, query_body = query_title.cuda(), query_body.cuda()
            similar_title, similar_body = similar_title.cuda(), similar_body.cuda()
            #TODO: for loop over random_title and random_body
        
        optimizer.zero_grad()
        model.hidden = model.init_hidden()

        lstm_query_title = model(query_title)
        lstm_query_body = model(query_body)
        lstm_query = (lstm_query_title + lstm_query_body)/2.0

        lstm_similar_title = model(similar_title)
        lstm_similar_body = model(similar_body)
        lstm_similar = (lstm_similar_title + lstm_similar_body)/2.0

        #TODO: for loop over random_title and random_body

        score_pos = cosine_similarity(lstm_query, lstm_similar)
        #TODO: score_neg

        loss = criterion([score_pos, score_neg], 1) #y=1
        loss.backward()
        optimizer.step()
                
        running_train_loss += loss.cpu().data[0]        
        
    #end for
    training_loss.append(running_train_loss)
    print "epoch: %4d, training loss: %.4f" %(epoch+1, running_train_loss)
    
    torch.save(model, SAVE_PATH)
#end for

"""
#generate plots
plt.figure()
plt.plot(training_loss, label='Adam')
plt.title("LSTM Model Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.legend()
plt.savefig('./figures/training_loss.png')

plt.figure()
plt.plot(validation_loss, label='Adam')
plt.title("LSTM Model Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Validation Loss")
plt.legend()
plt.savefig('./figures/validation_loss.png')
"""

In [ ]:
import configparser
config = configparser.ConfigParser()
config['DEFAULT'] = {'ServerAliveInterval': '45',
                     'Compression': 'yes',
                     'CompressionLevel': '9'}
config['bitbucket.org'] = {}
config['bitbucket.org']['User'] = 'hg'
config['topsecret.server.com'] = {}
topsecret = config['topsecret.server.com']
topsecret['Port'] = '50022'     # mutates the parser
topsecret['ForwardX11'] = 'no'  # same here
config['DEFAULT']['ForwardX11'] = 'yes'
with open('example.ini', 'w') as configfile:
  config.write(configfile)